In [4]:
import pandas as pd
import numpy as np 
import plotly.express as px
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import re

In [5]:
df=pd.read_csv("Advertising.csv")

In [6]:
df

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [27]:
n =df["TV"].sum() 
i=df["Sales"].sum()

In [30]:
n

29408.5

In [38]:
df.corr()

,Unnamed: 0,TV,Radio,Newspaper,Sales
Unnamed: 0,1.000000,0.017715,-0.110680,-0.154944,-0.051616
TV,0.017715,1.000000,0.054809,0.056648,0.782224
Radio,-0.110680,0.054809,1.000000,0.354104,0.576223
Newspaper,-0.154944,0.056648,0.354104,1.000000,0.228299
Sales,-0.051616,0.782224,0.576223,0.228299,1.000000


NO COLUMN IS CORRELATED 80% 

In [31]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [36]:
correlation(df , 0.80)

set()

Trying to find outliers with iqr

In [43]:
def detect_outliers(data):
    outlier_percents = {}
    for column in data.columns:
        if data[column].dtype != object:
            q1 = np.quantile(data[column], 0.25)
            q3 = np.quantile(data[column], 0.75)
            iqr = q3 - q1
            upper__bound = q3 + (1.5 * iqr)
            lower__bound = q1 - (1.5 * iqr)
            outliers = data[(data[column] > upper__bound) | (data[column] < lower__bound)][column]
            outlierpercentage = len(outliers) / len(data[column]) * 100
            outlier_percents[column] = outlierpercentage
            outlier_dataframe = pd.DataFrame(data = outlier_percents.values() ,index=outlier_percents.keys() ,columns=['Outlier_percentage'])
    
    return outlier_dataframe.sort_values(by = 'Outlier_percentage', ascending = False)

detect_outliers(df)

,Outlier_percentage
Newspaper,1.0
Unnamed: 0,0.0
TV,0.0
Radio,0.0
Sales,0.0


In [45]:
X=df[['TV','Radio','Newspaper']]
y=df['Sales']

In [44]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20)

In [106]:
from sklearn.ensemble import RandomForestRegressor

In [117]:
regr = RandomForestRegressor(max_depth=5, random_state=0)

In [118]:
regr.fit(X, y)

RandomForestRegressor(max_depth=5, random_state=0)

In [119]:
pre = regr.predict(X_test)

In [120]:
from sklearn.metrics import r2_score

In [122]:
acc = r2_score(y_test, pre)
print("RandomForestRegressor model accuracy (in %):", acc*100)

RandomForestRegressor model accuracy (in %): 99.16238212761357
